# Compulsory 3 - Erlend Risvik


## Imports and loading data

In [8]:
import pandas as pd
from pyspark.sql import SparkSession
import os

os.environ["PYSPARK_PYTHON"] = "python"

spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').getOrCreate()

(spark.read.format("org.apache.spark.sql.cassandra")
 .options(table="fish_data_full", keyspace="compulsory")
 .load()
 .createOrReplaceTempView("fish_data_full"))

fish_data = spark.sql("SELECT * FROM fish_data_full").toPandas()

(spark.read.format("org.apache.spark.sql.cassandra")
 .options(table="lice_data_full", keyspace="compulsory")
 .load()
 .createOrReplaceTempView("lice_data_full"))

lice_data = spark.sql("SELECT * FROM lice_data_full").toPandas()

In [9]:
fish_data.head()

,localityweekid,avgadultfemalelice,hascleanerfishdeployed,hasila,hasmechanicalremoval,haspd,hasreportedlice,hassalmonoids,hassubstancetreatments,infilteredselection,...,isonland,isslaughterholdingcage,lat,localityno,lon,municipality,municipalityno,name,week,year
0,190827,NaN,False,False,False,False,False,True,False,True,...,False,False,69.881302,10753,19.434532,Karlsøy,1936,Strandmo,43,2015
1,140632,NaN,False,False,False,False,False,True,False,True,...,False,False,66.298866,13522,12.813833,Nesna,1828,Valvika,17,2015
2,913528,NaN,False,False,False,False,False,False,False,True,...,False,False,70.242447,34077,19.500000,Karlsøy,1936,Torsvåg II,25,2015
3,189850,NaN,False,False,False,False,False,True,False,True,...,False,False,68.868980,11414,15.301050,Øksnes,1868,Toften,19,2015
4,201349,0.4,False,False,False,False,True,True,False,True,...,False,False,60.885334,13567,5.461600,Masfjorden,1266,"Knappen,Solheim",2,2015


In [10]:
lice_data.head()

,id,avgadultfemalelice,avgmobilelice,avgstationarylice,hasbathtreatment,hascleanerfishdeployed,hasinfeedtreatment,hasmechanicalremoval,hasreportedlice,hassalmonoids,isfallow,isslaughterholdingcage,localityno,seatemperature,week,year
0,187305,0.10,0.10,0.50,True,True,False,False,True,True,False,False,24175,9.2,42,2015
1,187281,0.00,0.02,0.13,False,False,False,False,True,True,False,False,24175,4.0,18,2015
2,187315,0.04,0.15,0.21,False,True,False,False,True,True,False,False,24175,5.2,52,2015
3,187271,0.00,0.00,0.00,False,False,False,False,True,True,False,False,24175,3.3,8,2015
4,187272,NaN,NaN,NaN,False,False,False,False,False,True,False,False,24175,3.3,9,2015


## Pivoting

In [21]:
# We start by creating  a variable called a variable region, splitting Norway into three regions:
# South - below 63.81 latitude, Middle - up to 68.85 latitude, North - above 68.85 latitude

fish_data['region'] = fish_data['lat'].apply(lambda x: 'South' if x < 63.81 else ('Middle' if x < 68.85 else 'North'))
fish_data[["lat", "region"]].head(10)

,lat,region
0,69.881302,North
1,66.298866,Middle
2,70.242447,North
3,68.868980,North
4,60.885334,South
5,68.783134,Middle
6,60.960419,South
7,68.716782,Middle
8,60.191017,South
9,59.044918,South
